In [39]:
#%pip install pycaret[full]

In [40]:
import pandas as pd
from pycaret.regression import *
import datetime

In [41]:
df = pd.read_csv('ibov.csv', sep=';')
df.drop(['date', 'open', 'high', 'low', 'close', 'date_import', 'ema_200p_diff', 'volume'], axis=1, inplace=True)
df['date_time'] = pd.to_datetime(df['date_time'])
df.dropna(how='any', axis=0, inplace=True)
df = df.sort_values(['date_time', 'symbol'],
              ascending = [True, True])
df.reset_index(inplace = True)
df.drop(['index'], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183104 entries, 0 to 183103
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   adj_close  183104 non-null  float64       
 1   symbol     183104 non-null  object        
 2   date_time  183104 non-null  datetime64[ns]
 3   ema_200p   183104 non-null  float64       
 4   rsi        183104 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 7.0+ MB


In [42]:
symbols = set(list(df['symbol']))
symbols

{'ABEV3.SA',
 'ALPA4.SA',
 'AMER3.SA',
 'ARZZ3.SA',
 'ASAI3.SA',
 'AZUL4.SA',
 'B3SA3.SA',
 'BBAS3.SA',
 'BBDC3.SA',
 'BBDC4.SA',
 'BBSE3.SA',
 'BEEF3.SA',
 'BPAC11.SA',
 'BPAN4.SA',
 'BRAP4.SA',
 'BRFS3.SA',
 'BRKM5.SA',
 'CASH3.SA',
 'CCRO3.SA',
 'CIEL3.SA',
 'CMIG4.SA',
 'CMIN3.SA',
 'COGN3.SA',
 'CPFE3.SA',
 'CPLE6.SA',
 'CRFB3.SA',
 'CSAN3.SA',
 'CSNA3.SA',
 'CVCB3.SA',
 'CYRE3.SA',
 'DXCO3.SA',
 'ECOR3.SA',
 'EGIE3.SA',
 'ELET3.SA',
 'ELET6.SA',
 'EMBR3.SA',
 'ENBR3.SA',
 'ENEV3.SA',
 'ENGI11.SA',
 'EQTL3.SA',
 'EZTC3.SA',
 'FLRY3.SA',
 'GGBR4.SA',
 'GOAU4.SA',
 'GOLL4.SA',
 'HAPV3.SA',
 'HYPE3.SA',
 'IGTI11.SA',
 'ITSA4.SA',
 'ITUB4.SA',
 'JBSS3.SA',
 'KLBN11.SA',
 'LREN3.SA',
 'LWSA3.SA',
 'MGLU3.SA',
 'MRFG3.SA',
 'MRVE3.SA',
 'MULT3.SA',
 'NTCO3.SA',
 'PCAR3.SA',
 'PETR3.SA',
 'PETR4.SA',
 'PETZ3.SA',
 'PRIO3.SA',
 'QUAL3.SA',
 'RADL3.SA',
 'RAIL3.SA',
 'RAIZ4.SA',
 'RDOR3.SA',
 'RENT3.SA',
 'RRRP3.SA',
 'SANB11.SA',
 'SBSP3.SA',
 'SLCE3.SA',
 'SMTO3.SA',
 'SOMA3.SA',
 'SUZB3

In [53]:
dataset = pd.DataFrame()
for s in symbols:
  aux = df[df['symbol'] == s]
  aux['adj_close_1'] = aux['adj_close'].shift()
  aux['adj_close_2'] = aux['adj_close'].shift(2)

  aux['rsi_1'] = aux['rsi'].shift()
  aux['rsi_2'] = aux['rsi'].shift(2)

  aux['ema_200p_1'] = aux['ema_200p'].shift()
  aux['ema_200p_2'] = aux['ema_200p'].shift(2)

  dataset = pd.concat([dataset, aux])

dataset.drop(['ema_200p', 'rsi'], axis=1, inplace=True)
dataset.dropna(how='any', axis=0, inplace=True)
dataset

,adj_close,symbol,date_time,adj_close_1,adj_close_2,rsi_1,rsi_2,ema_200p_1,ema_200p_2
136,11.882272,ABEV3.SA,2013-10-21,11.900176,11.843707,58.055147,56.267613,11.430742,11.425286
204,11.955268,ABEV3.SA,2013-10-22,11.882272,11.900176,57.256041,58.055147,11.435922,11.430742
272,11.638481,ABEV3.SA,2013-10-23,11.955268,11.882272,59.692065,57.256041,11.441871,11.435922
340,11.707349,ABEV3.SA,2013-10-24,11.638481,11.955268,47.136964,59.692065,11.444120,11.441871
408,11.777594,ABEV3.SA,2013-10-25,11.707349,11.638481,49.617886,47.136964,11.447126,11.444120
...,...,...,...,...,...,...,...,...,...
182724,34.310001,PETR3.SA,2023-06-20,34.220001,33.389999,82.001335,79.749005,24.986540,24.893741
182812,35.680000,PETR3.SA,2023-06-21,34.310001,34.220001,82.232100,82.001335,25.079310,24.986540
182900,35.090000,PETR3.SA,2023-06-22,35.680000,34.310001,85.317972,82.232100,25.184790,25.079310
182988,33.900002,PETR3.SA,2023-06-23,35.090000,35.680000,78.958009,85.317972,25.283349,25.184790


In [54]:
categorial_features = ['symbol']
numeric_features = ['adj_close_1', 'adj_close_2', 'ema_200p_1', 'ema_200p_1', 'rsi_1', 'rsi_2']
date_features = ['date_time']
exp_name = setup(data = dataset,
                 target = 'adj_close',
                 categorical_features=categorial_features,
                 numeric_features=numeric_features,
                 date_features=date_features,
                 fold_strategy = 'timeseries',
                 fold = 3,
                 transform_target = True,
                 session_id=123,
                 use_gpu = True,
                 index = False)

,Description,Value
0,Session id,123
1,Target,adj_close
2,Target type,Regression
3,Original data shape,"(182928, 9)"
4,Transformed data shape,"(182928, 26)"
5,Transformed train set shape,"(128049, 26)"
6,Transformed test set shape,"(54879, 26)"
7,Numeric features,6
8,Date features,1
9,Categorical features,1


In [ ]:
best = compare_models(sort = 'MAE')

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,17:14:35
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
dt,Decision Tree Regressor,0.4543,0.6866,0.8276,0.9965,0.0374,0.0286,4.5233
knn,K Neighbors Regressor,0.7052,1.1614,1.0751,0.9940,0.0768,0.0701,15.5467
lr,Linear Regression,3.4991,108.4416,10.4061,0.4445,0.2664,0.3671,0.7233
br,Bayesian Ridge,3.5042,108.0919,10.3896,0.4463,0.2666,0.3676,0.6367
ridge,Ridge Regression,3.5074,107.9310,10.3819,0.4471,0.2668,0.3678,0.5367
llar,Lasso Least Angle Regression,4.1597,138.8121,11.7753,0.2889,0.3275,0.5587,0.4733
lasso,Lasso Regression,4.1602,139.1360,11.7889,0.2873,0.3275,0.5587,0.8667
omp,Orthogonal Matching Pursuit,4.2624,212.9774,14.5853,-0.0910,0.3099,0.4991,0.4667
par,Passive Aggressive Regressor,4.2625,261.4298,16.1415,-0.3385,0.3378,0.5692,0.9200
en,Elastic Net,4.2916,174.1717,13.1914,0.1077,0.3205,0.5379,3.4667


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

In [48]:
prediction_holdout = predict_model(best);

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extra Trees Regressor,0.2772,0.3816,0.6178,0.9981,0.0231,0.0170


In [49]:
prediction_holdout

,symbol,date_time,ema_200p,rsi,adj_close_1,adj_close_2,adj_close,prediction_label
128049,YDUQ3.SA,2018-09-12,23.766363,32.182224,18.446714,18.757116,3.701965,18.077907
128050,BBDC3.SA,2020-10-02,15.141553,39.281883,12.973660,12.952095,3.219759,12.902221
128051,ELET6.SA,2016-06-27,7.507384,82.393669,11.463313,11.368742,3.054620,11.646611
128052,SMTO3.SA,2013-12-26,7.426834,50.000938,7.407465,7.391737,2.525627,7.413857
128053,PCAR3.SA,2019-04-16,81.830986,66.631386,86.030815,86.030815,6.285553,86.030831
...,...,...,...,...,...,...,...,...
182923,UGPA3.SA,2015-04-07,24.087521,68.166107,29.157261,28.841190,4.383443,29.237965
182924,EZTC3.SA,2019-07-29,20.384577,79.099945,29.453371,28.524624,4.430195,30.315164
182925,SOMA3.SA,2021-10-18,15.592989,50.402313,16.962708,16.629335,3.574930,16.890973
182926,COGN3.SA,2023-03-20,2.405158,49.920433,2.230000,2.260000,1.252261,2.210949


In [51]:
# plot the data and MA
import plotly.express as px
data = prediction_holdout[prediction_holdout['symbol'] == 'PETR4.SA']
#data.info()
fig = px.line(data, x="date_time", y=['adj_close'], template = 'plotly_dark')
fig.show()

In [15]:
# generate predictions on the original dataset
predictions = predict_model(best, data=df)

# add a date column in the dataset
#predictions['Date'] = pd.date_range(start='1949-01-01', end = '1960-12-01', freq = 'MS')
# line plot
# fig = px.line(predictions, x='date_time', y=["adj_close", "Label"], template = 'plotly_dark')
# add a vertical rectange for test-set separation
# fig.add_vrect(x0="1960-01-01", x1="1960-12-01", fillcolor="grey", opacity=0.25, line_width=0)
# fig.show()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.4019,0.5274,0.7262,0.9974,0.0333,0.0284


In [16]:
predictions

,date,open,high,low,close,volume,symbol,date_time,date_import,ema_200p,ema_200p_diff,rsi,adj_close,prediction_label
0,2020-11-13,21.000000,21.350000,20.799999,20.799999,470000.0,RRRP3.SA,2020-11-13,2023-06-27 02:03:22.327851+00:00,NaN,NaN,NaN,3.676736,23.493202
1,2020-11-16,21.150000,21.250000,20.969999,21.000000,175800.0,RRRP3.SA,2020-11-16,2023-06-27 02:03:22.327851+00:00,NaN,NaN,NaN,3.689614,23.493202
2,2020-11-17,20.940001,20.990000,20.650000,20.990000,256600.0,RRRP3.SA,2020-11-17,2023-06-27 02:03:22.327851+00:00,NaN,NaN,NaN,3.688972,23.493202
3,2020-11-18,20.990000,21.070000,20.900000,20.900000,96600.0,RRRP3.SA,2020-11-18,2023-06-27 02:03:22.327851+00:00,NaN,NaN,NaN,3.683188,23.493202
4,2020-11-19,21.000000,21.040001,20.820000,20.900000,84300.0,RRRP3.SA,2020-11-19,2023-06-27 02:03:22.327851+00:00,NaN,NaN,NaN,3.683188,23.202965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200611,2023-06-20,18.930000,19.459999,18.709999,19.459999,5030300.0,YDUQ3.SA,2023-06-20,2023-06-27 02:03:22.327851+00:00,11.891008,63.650002,82.895332,3.587643,18.718752
200612,2023-06-21,19.629999,21.180000,19.549999,20.350000,12077100.0,YDUQ3.SA,2023-06-21,2023-06-27 02:03:22.327851+00:00,11.975178,69.930000,85.004684,3.647373,20.525383
200613,2023-06-22,20.100000,20.270000,19.080000,20.030001,6985900.0,YDUQ3.SA,2023-06-22,2023-06-27 02:03:22.327851+00:00,12.055326,66.150002,81.130653,3.626156,20.504188
200614,2023-06-23,19.540001,19.690001,17.920000,18.430000,11726100.0,YDUQ3.SA,2023-06-23,2023-06-27 02:03:22.327851+00:00,12.118754,52.080002,65.144234,3.515563,18.356157


In [17]:
plot_model(xgboost, plot = 'auc')


NameError: ignored

In [ ]:
plot_model(xgboost, plot = 'confusion_matrix')


In [ ]:
plot_model(xgboost, plot = 'feature')